__Author: Стурейко Игорь__

__Project: Geekbrains.NeutralNets__

__Lesson 1 - Основы Keras__

__Date: 2021-12-07__

# Импорты

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from keras.datasets import fashion_mnist  # набор данных


In [2]:
(train_imagef,train_labelf),(test_imagef,test_labelf)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [3]:
# Normalize the images.
train_imagef = (train_imagef / 127) - 1
test_imagef = (test_imagef / 127) - 1

# Flatten the images.
train_images = train_imagef.reshape((-1, 784))
test_images = test_imagef.reshape((-1, 784))

print(train_images.shape) # (60000, 784)
print(test_images.shape)  # (10000, 784)

(60000, 784)
(10000, 784)


In [4]:
# посмотрим сколько итоговых категорий
np.unique(train_labelf)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [5]:
# гиперпараметры модели
n_cat: int # number of output categories
n_layers: int # number of hidden layers
n_neurons: int # number of neurons in hidden layer
act_func: str # activation function
optimizer: str # optimizer
loss: str # loss function
metrics: str # metrics

In [6]:
class OptimizedModel:
  """
  Class for tests and optimisation model hyperparameters
  parameters:
  n_layers: int - number of hidden layers
  n_neurons: int - number of neurons in one hidden layer
  act_func: str - activation function on hidden layer
  optimizer: str - optimizer
  loss: str - loss function
  metrics: str - result metric
  epochs: int - number of epochs
  verbose: bool - verbose mode
  """
  def __init__(self, n_layers: int, n_neurons: int, act_func: str, optimizer: str, loss: str, metrics: str, epochs: int, verbose: bool):
    self.n_layers = n_layers
    self.n_neurons = n_neurons
    self.act_func = act_func
    self.optimizer = optimizer
    self.loss = loss
    self.metrics = metrics
    self.epochs = epochs
    self.verbose = verbose

  def calculate(self, train_images, test_images, train_labels, test_labels):
    n_cat = len(np.unique(train_labels)) # число выходных классов
    model1 = Sequential(name='fashon_model')
    model1.add(Dense( 128, activation='relu', input_shape=(784,)))  
    # входной слой + поносвязный слой из 128 нейронов с активацией ReLU

    for k in range(self.n_layers+1):
      model1.add(Dense(self.n_neurons, activation=self.act_func)) 
      # Скрытый слой k из 10+(10-k)*10 нейронов с активацией tanh

    model1.add(Dense(n_cat, activation='softmax')) 
    # выходной слой из 10 нейронов (сколько классов, столько нейронов) + активация softmax

    model1.compile(
      optimizer=self.optimizer, # оптимизатор
      loss=self.loss, # функция потерь
      metrics=[self.metrics], # метрика
    ) 

    y_train = to_categorical(train_labels)
    y_test = to_categorical(test_labels)

    # Train the model.
    model1.fit(  
      train_images[:,:],
      y_train[:,:],
      epochs=self.epochs,
      batch_size=1000, validation_split=0.2,
      verbose=self.verbose
    )

    # Evaluate the model.

    results = model1.evaluate(test_images, y_test, batch_size=128, verbose=False)
    return results[1]

In [7]:
res = []
for n in np.arange(10):
  mod = OptimizedModel(1, 120, 'tanh', 'adam', 'categorical_crossentropy', 'accuracy', 15, False)
  res.append(mod.calculate(train_images, test_images, train_labelf, test_labelf))
print(res)
print(np.round(np.mean(res), 4))

[0.8748000264167786, 0.880299985408783, 0.8726000189781189, 0.8805000185966492, 0.8773999810218811, 0.8805999755859375, 0.8822000026702881, 0.8813999891281128, 0.8770999908447266, 0.8790000081062317]
0.8786


In [8]:
res_p = pd.DataFrame(columns=['num_neutrons', 'accurancy'])
for n_neurons in np.arange(100, 200, 10):
  res = []
  for n in np.arange(10):
    mod = OptimizedModel(1, n_neurons, 'tanh', 'adam', 'categorical_crossentropy', 'accuracy', 15, False)
    res.append(mod.calculate(train_images, test_images, train_labelf, test_labelf))
  res_p.loc[len(res_p)] = [n_neurons, np.mean(res)]
res_p

,num_neutrons,accurancy
0,100.0,0.87634
1,110.0,0.87820
2,120.0,0.87806
3,130.0,0.87936
4,140.0,0.87792
5,150.0,0.87815
6,160.0,0.87727
7,170.0,0.87950
8,180.0,0.87989
9,190.0,0.87807


In [ ]:
res_p.to_csv('number_neutrons', encoding='utf-8')

## Практическое задание

Используем набор примеров fashion-MNIST

1.  Опишите - какой результата получен в нейросети в зависимости от:
  -  числа нейронов в слое(для 2-хслойной сети), 
  - числа слоев (2, 3, 5, 10) при близких размерах сети (близкое число тренируемых парметров).
  - фиксируйте для тренировочного и тестового набора метрики accuracy.
2.  Проверьте работу разных оптимизаторов (SGD, Adam, RMSProp) для одной из моделей п.1.Фиксируйте для тренировочного и тестового набора метрики accuracy.

3. Сделайте вывод - что помогло вам улучшить качество классификации в нейросети на тестовом наборе? 

4. Для одного варианта сетей сформируйте матрицу ошибок по классам. Оцените качество модели по каждому классу отдельно (полнота , точность). Сделайте вывод.